In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
from xgboost.sklearn import XGBRegressor

In [3]:
import numpy as np
import pandas as pd

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [5]:
train = pd.read_csv('updated_train.csv')
test = pd.read_csv('updated_test.csv')

In [6]:
df = pd.read_csv('fraction.csv')

In [7]:
features = ['vendor_id','tolls_amount', 'tip_amount', 'mta_tax','passenger_count','rate_code','surcharge',
            'total_jrny', 'CRD','CSH', 'DIS', 'NOC', 'UNK', 'distance']

In [8]:
X = df[features]
y = df['fare_amount']

# Model 1:- Random Forest

In [17]:
param_test1 = {
 'max_depth':[1,5,10,20,40,80],
  'min_samples_split':[2,10,20,50,100]  
}
gsearch1 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=111),
                        param_grid = param_test1,n_jobs=4,iid=False, cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 40, 'min_samples_split': 20}, 0.96108700642583234)

In [18]:
param_test2 = {
 'max_depth':[30,40,50,60],
  'min_samples_split':[15,20,25,30,50]  
}
gsearch2 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=111),
                        param_grid = param_test2,n_jobs=4,iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 30, 'min_samples_split': 15}, 0.96123270370460379)

In [19]:
param_test3 ={
    'min_samples_split':[11,12,13,14,15]
}
gsearch3 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=100, max_depth=30,min_samples_split=2,
                                                          min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=111),
                        param_grid = param_test3,n_jobs=4,iid=False, cv=5)
gsearch3.fit(X,y)
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_split': 15}, 0.96123270370460379)

In [20]:
param_test4 ={
    'min_samples_leaf':[1,2,5,10,15]
}
gsearch4 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=100, max_depth=30,min_samples_split=15,
                                                          min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=111),
                        param_grid = param_test4,n_jobs=4,iid=False, cv=5)
gsearch4.fit(X,y)
gsearch4.best_params_, gsearch4.best_score_

({'min_samples_leaf': 1}, 0.96123270370460379)

In [21]:
param_test5 ={
    'n_estimators':[100,200,300,400,500,1000]
}
gsearch5 = GridSearchCV(estimator = RandomForestRegressor(n_estimators=100, max_depth=30,min_samples_split=15,
                                                          min_samples_leaf=1,bootstrap=True,n_jobs=-1,random_state=111),
                        param_grid = param_test5,n_jobs=4,iid=False, cv=5)
gsearch5.fit(X,y)
gsearch5.best_params_, gsearch5.best_score_

({'n_estimators': 1000}, 0.96163183634812444)

In [25]:
rfr = RandomForestRegressor(n_estimators=500,max_depth=30, min_samples_split=15, min_samples_leaf=1, 
                            bootstrap=True,n_jobs=-1, random_state=111)

In [26]:
rfr.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=30,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=15, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=-1, oob_score=False, random_state=111,
           verbose=0, warm_start=False)

In [27]:
rfrpred = rfr.predict(test[features])

In [28]:
rfrpred = np.square(rfrpred)

In [29]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = rfrpred
submission.to_csv('rfr.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.799960
1,AJX000119872,7.579528
2,AJX000119873,10.521470
3,AJX000119874,19.113546
4,AJX000119875,15.979919


# Model 2:- XgBoost

In [39]:
param_test1 = {
 'max_depth':[3,5,7,9],
 'min_child_weight':[1,3,5]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=2,
 min_child_weight=1, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test1,n_jobs=4,iid=False, cv=10)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 7, 'min_child_weight': 3}, 0.96516659102429025)

In [40]:
# parameter testing first
param_test2 = {
 'max_depth':[6,7,8],
 'min_child_weight':[3,4,5]
}
gsearch2 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=2,
 min_child_weight=1, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test2,n_jobs=4,iid=False, cv=10)
gsearch2.fit(X,y)
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 7, 'min_child_weight': 3}, 0.96516659102429025)

In [41]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test3,n_jobs=4,iid=False, cv=10)
gsearch3.fit(X,y)
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.4}, 0.96532949710224225)

In [42]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.4, colsample_bylevel=0.85, colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(X,y)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.6, 'subsample': 0.9}, 0.96532035017636986)

In [44]:
param_test4 = {
 'subsample':[0.85,0.9,0.95],
 'colsample_bytree':[0.55,0.6,0.65]
}
gsearch4 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.4, colsample_bylevel=0.85, colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(X,y)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.65, 'subsample': 0.95}, 0.9653611413901324)

In [45]:
param_test = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.4, colsample_bylevel=0.85, colsample_bytree=0.65,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.95,reg_alpha=0,reg_lambda=1),
 param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 1e-05}, 0.9654499416727369)

In [46]:
param_test = {
 'reg_alpha':[1e-5,0, 0.001, 0.005, 0.01, 0.05]
}
gsearch = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.4, colsample_bylevel=0.85, colsample_bytree=0.65,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.95,reg_alpha=0,reg_lambda=1),
 param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'reg_alpha': 1e-05}, 0.9654499416727369)

In [47]:
param_test = {
 'colsample_bylevel':[i/10.0 for i in range(6,10)]
}
gsearch = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=7,
 min_child_weight=3, gamma=0.4, colsample_bylevel=0.85, colsample_bytree=0.65,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.95,reg_alpha=1e-5,reg_lambda=1),
 param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'colsample_bylevel': 0.8}, 0.9654499416727369)

In [121]:
xgb = XGBRegressor(learning_rate =0.01, n_estimators=500, max_depth=7,
 min_child_weight=5, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.65,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.95,reg_alpha=1e-5)

In [122]:
xgb.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.85,
       colsample_bytree=0.65, gamma=0.005, learning_rate=0.01,
       max_delta_step=0, max_depth=7, min_child_weight=5, missing=None,
       n_estimators=500, nthread=4, objective='reg:linear',
       reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=27,
       silent=True, subsample=0.95)

In [123]:
xgbpred = xgb.predict(test[features])

In [124]:
xgbpred = np.square(xgbpred)

In [126]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = xgbpred
submission.to_csv('xgb.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.948950
1,AJX000119872,7.616688
2,AJX000119873,10.464274
3,AJX000119874,19.255938
4,AJX000119875,15.102335


# Model 3:- KNN Model

In [129]:
from sklearn.neighbors import KNeighborsRegressor

In [131]:
param_test={
    'n_neighbors':list(range(1,31,1))
}
gsearch = GridSearchCV(estimator = KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                       metric='minkowski', n_jobs=-1),
param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'n_neighbors': 6}, 0.94422661142786635)

In [132]:
param_test={
    'weights':['uniform','distance']
}
gsearch = GridSearchCV(estimator = KNeighborsRegressor(n_neighbors=6, weights='uniform', algorithm='auto', leaf_size=30, p=2, 
                       metric='minkowski', n_jobs=-1),
param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'weights': 'distance'}, 0.94430016501764968)

In [134]:
param_test={
    'leaf_size':[10,20,30,40,50,60,70,80,90,100]
}
gsearch = GridSearchCV(estimator = KNeighborsRegressor(n_neighbors=6, weights='distance', algorithm='auto', leaf_size=30, p=2, 
                       metric='minkowski', n_jobs=-1),
param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'leaf_size': 10}, 0.94431646022403748)

In [135]:
param_test={
    'p':[1,2]
}
gsearch = GridSearchCV(estimator = KNeighborsRegressor(n_neighbors=6, weights='distance', algorithm='auto', leaf_size=10, p=2, 
                       metric='minkowski', n_jobs=-1),
param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'p': 1}, 0.94675829071020079)

In [181]:
knn = KNeighborsRegressor(n_neighbors=12, weights='uniform', algorithm='auto', leaf_size=10, p=1, 
                       metric='minkowski', n_jobs=-1)

In [182]:
knn.fit(X,y)

KNeighborsRegressor(algorithm='auto', leaf_size=10, metric='minkowski',
          metric_params=None, n_jobs=-1, n_neighbors=12, p=1,
          weights='uniform')

In [183]:
knnpred = knn.predict(test[features])

In [184]:
knnpred = np.square(knnpred)

In [185]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = knnpred
submission.to_csv('knn.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.963915
1,AJX000119872,7.803792
2,AJX000119873,10.579403
3,AJX000119874,18.611164
4,AJX000119875,15.123788


# Model 4:- ExtraTree Regressor

In [193]:
param_test = {
 'max_depth':[1,5,10,20,40,80],
  'min_samples_split':[2,10,20,50,100]  
}
gsearch = GridSearchCV(estimator = ExtraTreesRegressor(n_estimators=10, max_depth=10, min_samples_split=2, 
                                                       min_samples_leaf=1,max_features='auto',max_leaf_nodes=None,
                                                       bootstrap=True, n_jobs=-1, random_state=111),
                       param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 80, 'min_samples_split': 10}, 0.95880477389779306)

In [194]:
param_test = {
 'max_depth':[80,100,120,140,160,200],
  'min_samples_split':[2,10,20,50,100]  
}
gsearch = GridSearchCV(estimator = ExtraTreesRegressor(n_estimators=10, max_depth=10, min_samples_split=2, 
                                                       min_samples_leaf=1,max_features='auto',max_leaf_nodes=None,
                                                       bootstrap=True, n_jobs=-1, random_state=111),
                       param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 80, 'min_samples_split': 10}, 0.95880477389779306)

In [195]:
param_test = {
 'min_samples_leaf':[1,5,10,15,20,25,50,100]
}
gsearch = GridSearchCV(estimator = ExtraTreesRegressor(n_estimators=10, max_depth=80, min_samples_split=10, 
                                                       min_samples_leaf=1,max_features='auto',
                                                       bootstrap=True, n_jobs=-1, random_state=111),
                       param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 1}, 0.95880477389779306)

In [196]:
param_test = {
 'max_features':['auto','sqrt','log2']
}
gsearch = GridSearchCV(estimator = ExtraTreesRegressor(n_estimators=10, max_depth=80, min_samples_split=10, 
                                                       min_samples_leaf=1,max_features='auto',
                                                       bootstrap=True, n_jobs=-1, random_state=111),
                       param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_features': 'auto'}, 0.95880477389779306)

In [227]:
ext = ExtraTreesRegressor(n_estimators=300, max_depth=100, min_samples_split=10, min_samples_leaf=1,max_features='auto',
                          bootstrap=True, n_jobs=-1, random_state=111)

In [228]:
ext.fit(X,y)

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=100,
          max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
          min_samples_split=10, min_weight_fraction_leaf=0.0,
          n_estimators=300, n_jobs=-1, oob_score=False, random_state=111,
          verbose=0, warm_start=False)

In [229]:
extpred = ext.predict(test[features])

In [230]:
extpred = np.square(extpred)

In [231]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = extpred
submission.to_csv('general.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.862577
1,AJX000119872,7.603246
2,AJX000119873,10.502739
3,AJX000119874,19.053924
4,AJX000119875,15.331628


# stacking or blending

In [232]:
ext = ExtraTreesRegressor(n_estimators=500, max_depth=80, min_samples_split=10, min_samples_leaf=1,max_features='auto',
                          bootstrap=True, n_jobs=-1, random_state=111)

In [233]:
xgb = XGBRegressor(learning_rate =0.01, n_estimators=500, max_depth=7,
 min_child_weight=5, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.65,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.95,reg_alpha=1e-5)

In [234]:
rfr = RandomForestRegressor(n_estimators=500,max_depth=30, min_samples_split=15, min_samples_leaf=1, 
                            bootstrap=True,n_jobs=-1, random_state=111)

In [235]:
new_df = train.sample(frac=0.03)

In [242]:
X_test = new_df[features]
y_test = np.sqrt(new_df['fare_amount'])

In [243]:
a = ext.predict(X_test)

In [244]:
b = rfr.predict(X_test)

In [245]:
c = xgb.predict(X_test)

In [252]:
X = pd.DataFrame(index = new_df.index)

In [253]:
X['extfare'] = a
X['rfrfare'] = b
X['xgbfare'] = c

In [256]:
X.head()

,extfare,rfrfare,xgbfare,fare_amount
1528714,2.960485,2.940240,2.863404,2.966479
1656227,3.807225,3.886794,3.863960,3.974921
189704,3.623491,3.529215,3.576431,3.464102
540034,3.743166,3.756355,3.781459,3.741657
169033,2.723346,2.698345,2.779093,2.645751


In [255]:
X['fare_amount'] = y_test

In [257]:
lin = LinearRegression()

In [258]:
features1 = ['extfare','rfrfare','xgbfare']

In [259]:
lin.fit(X[features1],X['fare_amount'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [260]:
u = ext.predict(test[features])

In [261]:
v = rfr.predict(test[features])

In [262]:
w = xgb.predict(test[features])

In [263]:
Y = pd.DataFrame()

In [264]:
Y['extfare'] = u
Y['rfrfare'] = v
Y['xgbfare'] = w

In [266]:
pred = lin.predict(Y[features1])

In [267]:
pred = np.square(pred)

In [268]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = pred
submission.to_csv('stacking.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.722882
1,AJX000119872,7.529343
2,AJX000119873,10.952937
3,AJX000119874,19.500471
4,AJX000119875,15.749650
